In [3]:
from langchain_core.tools import tool, StructuredTool
from typing import Optional


@tool
def multiply(a: Optional[int], b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'title': 'A'}, 'b': {'title': 'B', 'type': 'integer'}}


---

In [1]:
"""Base class for Google Calendar tools."""

from __future__ import annotations

from typing import TYPE_CHECKING

from langchain_core.tools import BaseTool
from pydantic import Field

from langchain_google_community.gmail.utils import build_resource_service

if TYPE_CHECKING:
    # This is for linting and IDE typehints
    from googleapiclient.discovery import Resource  # type: ignore[import]
else:
    try:
        # We do this so pydantic can resolve the types when instantiating
        from googleapiclient.discovery import Resource
    except ImportError:
        pass

class GoogleCalendarBaseTool(BaseTool):
    api_resource: Resource = Field(default_factory=build_resource_service)
    
    @classmethod
    def from_api_resource(cls, api_resource: Resource) -> "GoogleCalendarBaseTool":
        """Create a tool from an api resource.

        Args:
            api_resource: The api resource to use.

        Returns:
            A tool.
        """
        return cls(api_resource=api_resource)

---

In [3]:
"""Create an event in Google Calendar.""" # VIEJO

from datetime import datetime
from typing import Any, Dict, List, Optional, Type, Union
from uuid import uuid4

from pydantic import BaseModel, Field, EmailStr

from langchain_core.callbacks import CallbackManagerForToolRun


def get_current_datetime() -> str:
    """Get the current datetime."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class CreateEventSchema(BaseModel):
    """Input for CalendarCreateEvent."""

    summary: str = Field(
        ...,
        description="The title of the event."
    )
    start_datetime: str = Field(
        default_factory=get_current_datetime,
        description=("The start datetime for the event in 'YYYY-MM-DD HH:MM:SS' format. "
                    f"The current year is {get_current_datetime()[:4]}. "
                    "If the event is all day type, then set the time to 'YYYY-MM-DD' format.")
    )
    end_datetime: str = Field(
        ...,
        description=("The end datetime for the event in 'YYYY-MM-DD HH:MM:SS' format"
                     "If the event is all day type, then set the time to 'YYYY-MM-DD' format, and it has to be one day after the start date.")
    )
    calendar_id: str = Field(
        default="primary", 
        description="The calendar id to create the event in."
    )
    timezone: Optional[str] = Field(
        default=None,
        description="The timezone of the event."
    )
    recurrence: Optional[Dict[str, Any]] = Field(
        default=None,
        description=("The recurrence of the event."
                     "The format is"
                     "{'FREQ': <'DAILY' or 'WEEKLY'>,"
                     "'INTERVAL': <number>,"
                     "'COUNT': <number or None>,"
                     "'UNTIL': <'YYYYMMDD' or None>,"
                     "'BYDAY': <'MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU' or None>}"
                     "Can be used COUNT or UNTIL, but not both, set the other to None.")
    )
    location: Optional[str] = Field(
        default=None,
        description="The location of the event."
    )
    description: Optional[str] = Field(
        default=None,
        description="The description of the event."
    )
    attendees: Optional[List[EmailStr]] = Field(
        default=None,
        description="The list of attendees' email addresses for the event."
    )
    reminders: Union[None, bool, List[Dict[str, Any]]] = Field(
        default=None,
        description=("The reminders for the event."
                     "If reminders are needed but are not specific, then set to 'True'"
                     "If specified, then set as [{'method': 'email', 'minutes': <minutes>}]"
                     "Or set as [{'method': 'popup', 'minutes': <minutes>}]"
                     "Where <minutes> is the number of minutes before the event."
                     "60 minutes = 1 hour."
                     "60 * 24 = 1 day.")
    )
    conference_data: Optional[bool] = Field(
        default=None,
        description="Whether to include conference data."
    )
    color_id: Optional[str] = Field(
        default=None,
        description=("The color id of the event, None is for default."
                     "'1': Lavender"
                     "'2': Sage"
                     "'3': Grape"
                     "'4': Flamingo"
                     "'5': Banana"
                     "'6': Tangerine"
                     "'7': Peacock"
                     "'8': Graphite"
                     "'9': Blueberry"
                     "'10': Basil"
                     "'11': Tomato")
    )


class CalendarCreateEvent(GoogleCalendarBaseTool):
    """Tool that create a event in Google Calendar."""

    name: str = "create_calendar_event"
    description: str = (
        "Use this tool to create an event." 
        "The input must be the summary, start and end datetime for the event."
    )
    args_schema: Type[CreateEventSchema] = CreateEventSchema

    def _get_timezone(self, calendar_id: str) -> str:
        """Get the timezone of the specified calendar."""
        calendars = self.api_resource.calendarList().list().execute().get('items', [])
        if not calendars:
            raise ValueError("No calendars found.")

        if calendar_id == 'primary':
            return calendars[0]['timeZone']
        else:
            for item in calendars:
                if item['id'] == calendar_id and item['accessRole'] != 'reader':
                    return item['timeZone']
            raise ValueError(f"Timezone not found for calendar ID: {calendar_id}")
            
    def _is_all_day_event(self, start_datetime: str, end_datetime: str) -> bool:
        """Check if the event is all day."""
        try:
            datetime.strptime(start_datetime, "%Y-%m-%d")
            datetime.strptime(end_datetime, "%Y-%m-%d")
            return True
        except ValueError:
            return False

    def _prepare_event(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        calendar_id: str = 'primary',
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None, 
        attendees: Optional[List[EmailStr]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conference_data: Optional[bool] = None, 
        color_id: Optional[str] = None
    ) -> Dict[str, Any]:
        """Prepare the event body."""
        timezone = timezone if timezone else self.__get_timezone(calendar_id)

        try:
            if self._is_all_day_event(start_datetime, end_datetime):
                start = {"date": start_datetime, "timeZone": timezone}
                end = {"date": end_datetime, "timeZone": timezone}
            else:
                date_object = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
                start = {"dateTime": date_object.astimezone().replace(microsecond=0).isoformat(), "timeZone": timezone}
                date_object = datetime.strptime(end_datetime, "%Y-%m-%d %H:%M:%S")
                end = {"dateTime": date_object.astimezone().replace(microsecond=0).isoformat(), "timeZone": timezone}
        except ValueError as error:
            raise ValueError("The datetime format is incorrect.") from error
        
        recurrence_data = None
        if recurrence:
            if isinstance(recurrence, dict):
                recurrence_data = ['RRULE:']
                for k, v in recurrence.items():
                    if v is not None:
                        recurrence_data.append(f"{k}={v};")
                recurrence_data = ''.join(recurrence_data)
        
        attendees_mails = [{"email": email} for email in attendees] if attendees else []
        
        reminders_info = None
        if reminders: 
            if reminders is True:
                reminders_info = {"useDefault": True}
            elif isinstance(reminders, list):
                for reminder in reminders:
                    if 'method' not in reminder or 'minutes' not in reminder:
                        raise ValueError("The reminders must have 'method' and 'minutes' keys.")
                    if reminder['method'] not in ['email', 'popup']:
                        raise ValueError("The reminders method must be 'email' or 'popup'.")
                reminders_info = {
                    'useDefault': False,
                    "overrides": reminders 
                }
        else:
            reminders_info = {"useDefault": False}
        
        if conference_data:
            conference_data = {
                "createRequest": {
                    "requestId": str(uuid4()),
                    "conferenceSolutionKey": {
                        "type": "hangoutsMeet"
                    }
                }
            }
        
        return {
            "summary": summary,
            "location": location,
            "description": description,
            "start": start,
            "end": end,
            "recurrence": [recurrence_data], 
            "attendees": attendees_mails, 
            "reminders": reminders_info,
            "conferenceData": conference_data, 
            "colorId": color_id
        }
    
    def _run(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        calendar_id: str = 'primary',
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None,
        attendees: Optional[List[EmailStr]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conference_data: Optional[bool] = None, 
        color_id: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Run the tool."""
        try:
            body = self._prepare_event(summary=summary, 
                                       start_datetime=start_datetime, 
                                       end_datetime=end_datetime, 
                                       calendar_id=calendar_id,
                                       timezone=timezone,
                                       recurrence=recurrence,
                                       location=location, 
                                       description=description, 
                                       attendees=attendees, 
                                       reminders=reminders,
                                       conference_data=conference_data, 
                                       color_id=color_id)
            
            conferenceVersion = 1 if conference_data else 0
            event = self.api_resource.events().insert(calendarId=calendar_id, 
                                                      body=body, 
                                                      conferenceDataVersion=conferenceVersion).execute()
            return event.get('htmlLink')
        except Exception as error:
            raise Exception(f"An error occurred: {error}")


In [172]:
"""Get the events in Google Calendar.""" # VIEJO

from datetime import datetime
from typing import Any, Dict, List, Optional

from pydantic import BaseModel, Field
from pytz import timezone

from langchain_core.callbacks import CallbackManagerForToolRun


def get_current_datetime() -> str:
    """Get the current datetime."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class SearchEventsSchema(BaseModel):
    """Input for CalendarGetEvents."""

    min_datetime: Optional[str] = Field(
        default_factory=get_current_datetime,
        description=(
            "The start datetime for the events in 'YYYY-MM-DD HH:MM:SS' format. "
            f"The current year is {datetime.now().year}."
        )
    )
    max_datetime: Optional[str] = Field(
        ...,
        description="The end datetime for the events in 'YYYY-MM-DD HH:MM:SS' format",
    )
    max_results: int = Field(
        default=10,
        description="The maximum number of results to return."
    )
    single_events: bool = Field(
        default=True,
        description="Whether to expand recurring events into instances and only return single one-off events and instances of recurring events."
    )
    order_by: str = Field(
        default="startTime",
        description="The order of the events, 'startTime' or 'updated'."
    )
    query: Optional[str] = Field(
        default=None,
        description=("Free text search terms to find events that match these terms in the following fields:"
                     "summary, description, location, attendee's displayName, attendee's email, organizer's displayName, organizer's email")
    )
    

class CalendarSearchEvents(GoogleCalendarBaseTool):
    """Tool that get the events in Google Calendar."""
    name: str = "search_events"
    description: str = "Use this tool to search the events in the calendar."
    args_schema: Type[SearchEventsSchema] = SearchEventsSchema


    def __get_calendars_info(self) -> List[Any]:
        """Get the calendars info."""
        calendars = self.api_resource.calendarList().list().execute()
        return calendars['items']
    
    def __get_calendar_timezone(self, calendars_info: List, calendar_id: str) -> Optional[str]:
        """Get the timezone of the current calendar."""
        for cal in calendars_info:
            if cal['id'] == calendar_id:
                return cal['timeZone']
        return None

    def __get_id_calendars(self, calendars_info: List) -> List[str]:
        """Get the calendars IDs."""
        calendars = []
        for cal in calendars_info:
            if cal.get('selected', None):
                calendars.append(cal['id'])
        return calendars
    
    def _process_data_events(self, events_data: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Process the data events."""
        simplified_data = []
        for data in events_data:
            # Extract relevant fields
            event_dict = {
            "id": data["id"],
            "htmlLink": data["htmlLink"],
            "summary": data["summary"],
            "creator": data["creator"]["email"],
            "organizer": data["organizer"]["email"],
            "start": data["start"]["dateTime"],
            "end": data["end"]["dateTime"],
            }
            simplified_data.append(event_dict)
        return simplified_data
    
    def _run(
        self,
        min_datetime: Optional[str],
        max_datetime: Optional[str] = None,
        max_results: int = 10,
        single_events: bool = True,
        order_by: str = "startTime",
        query: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> List[Dict[str, str]]:
        """Run the tool."""
        try:
            # body_request = self._prepare_request(min_datetime, max_datetime, max_results)
            calendars_info = self.__get_calendars_info()
            calendars = self.__get_id_calendars(calendars_info)
            events = []
            timeMin = None
            timeMax = None
            for calendar in calendars:
                region_tz = timezone(self.__get_calendar_timezone(calendars_info, calendar))
                if min_datetime:
                    timeMin = region_tz.localize(datetime.strptime(min_datetime, "%Y-%m-%d %H:%M:%S")).isoformat()
                if max_datetime:
                    timeMax = region_tz.localize(datetime.strptime(max_datetime, "%Y-%m-%d %H:%M:%S")).isoformat()
                events_result = self.api_resource.events().list(
                    calendarId=calendar, 
                    timeMin=timeMin, 
                    timeMax=timeMax, 
                    maxResults=max_results, 
                    singleEvents=single_events, 
                    orderBy=order_by,
                    q=query
                ).execute()
                cal_events = events_result.get('items', [])
                events.extend(cal_events)
            return self._process_data_events(events)
        except Exception as error:
            raise Exception(f"An error occurred: {error}")

---

In [10]:
"""Create an event in Google Calendar.""" # NUEVO

from datetime import datetime
from typing import Any, Dict, List, Optional, Type, Union
from uuid import uuid4
import re

from pydantic import BaseModel, Field

from langchain_core.callbacks import CallbackManagerForToolRun


def get_current_datetime() -> str:
    """Get the current datetime."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


class CreateEventSchema(BaseModel):
    """Input for CalendarCreateEvent."""

    summary: str = Field(
        ...,
        description="The title of the event."
    )
    start_datetime: str = Field(
        default_factory=get_current_datetime,
        description=(
            "The start datetime for the event in 'YYYY-MM-DD HH:MM:SS' format. "
            f"The current year is {datetime.now().year}. "
            "If the event is an all-day event, set the time to 'YYYY-MM-DD' format."
        )
    )
    end_datetime: str = Field(
        ...,
        description=(
            "The end datetime for the event in 'YYYY-MM-DD HH:MM:SS' format. "
            "If the event is an all-day event, set the time to 'YYYY-MM-DD' format."
        )
    )
    calendar_id: str = Field(
        default="primary",
        description="The calendar ID to create the event in."
    )
    timezone: Optional[str] = Field(
        default=None,
        description="The timezone of the event."
    )
    recurrence: Optional[Dict[str, Any]] = Field(
        default=None,
        description=(
            "The recurrence of the event. "
            "Format: {'FREQ': <'DAILY' or 'WEEKLY'>, 'INTERVAL': <number>, "
            "'COUNT': <number or None>, 'UNTIL': <'YYYYMMDD' or None>, "
            "'BYDAY': <'MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU' or None>}. "
            "Use either COUNT or UNTIL, but not both; set the other to None."
        )
    )
    location: Optional[str] = Field(
        default=None,
        description="The location of the event."
    )
    description: Optional[str] = Field(
        default=None,
        description="The description of the event."
    )
    attendees: Optional[List[str]] = Field(
        default=None,
        description="A list of attendees' email addresses for the event."
    )
    reminders: Union[None, bool, List[Dict[str, Any]]] = Field(
        default=None,
        description=(
            "Reminders for the event. "
            "Set to True for default reminders, or provide a list like "
            "[{'method': 'email', 'minutes': <minutes>}, ...]. "
            "Valid methods are 'email' and 'popup'."
        )
    )
    conference_data: Optional[bool] = Field(
        default=None,
        description="Whether to include conference data."
    )
    color_id: Optional[str] = Field(
        default=None,
        description=(
            "The color ID of the event. None for default. "
            "'1': Lavender, '2': Sage, '3': Grape, '4': Flamingo, '5': Banana, "
            "'6': Tangerine, '7': Peacock, '8': Graphite, '9': Blueberry, "
            "'10': Basil, '11': Tomato."
        )
    )
    transparency: Optional[str] = Field(
        default=None, 
        description=(
            "User availability for the event."
            "transparent for available and opaque for busy."
        )
    )


class CalendarCreateEvent(GoogleCalendarBaseTool):
    """Tool that creates an event in Google Calendar."""

    name: str = "create_calendar_event"
    description: str = (
        "Use this tool to create an event. "
        "The input must include the summary, start, and end datetime for the event."
    )
    args_schema: Type[CreateEventSchema] = CreateEventSchema

    def _get_timezone(self, calendar_id: str) -> str:
        """Get the timezone of the specified calendar."""
        calendars = self.api_resource.calendarList().list().execute().get('items', [])
        if not calendars:
            raise ValueError("No calendars found.")

        if calendar_id == 'primary':
            return calendars[0]['timeZone']
        else:
            for item in calendars:
                if item['id'] == calendar_id and item['accessRole'] != 'reader':
                    return item['timeZone']
            raise ValueError(f"Timezone not found for calendar ID: {calendar_id}")

    def _is_all_day_event(self, start_datetime: str, end_datetime: str) -> bool:
        """Check if the event is an all-day event."""
        date_format = "%Y-%m-%d"
        try:
            datetime.strptime(start_datetime, date_format)
            datetime.strptime(end_datetime, date_format)
            return True
        except ValueError:
            return False

    def _prepare_event(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        calendar_id: str = 'primary',
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None,
        attendees: Optional[List[str]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conference_data: Optional[bool] = None,
        color_id: Optional[str] = None, 
        transparency: Optional[str] = None
    ) -> Dict[str, Any]:
        """Prepare the event body."""
        timezone = timezone or self._get_timezone(calendar_id)

        try:
            if self._is_all_day_event(start_datetime, end_datetime):
                start = {"date": start_datetime}
                end = {"date": end_datetime}
            else:
                datetime_format = "%Y-%m-%d %H:%M:%S"
                start_dt = datetime.strptime(start_datetime, datetime_format)
                end_dt = datetime.strptime(end_datetime, datetime_format)
                start = {
                    "dateTime": start_dt.astimezone().isoformat(),
                    "timeZone": timezone
                }
                end = {
                    "dateTime": end_dt.astimezone().isoformat(),
                    "timeZone": timezone
                }
        except ValueError as error:
            raise ValueError("The datetime format is incorrect.") from error

        recurrence_data = None
        if recurrence:
            if isinstance(recurrence, dict):
                recurrence_items = [f"{k}={v}" for k, v in recurrence.items() if v is not None]
                recurrence_data = 'RRULE:' + ';'.join(recurrence_items)

        attendees_emails = []
        if attendees:
            email_pattern = r'^[^@]+@[^@]+\.[^@]+$'
            for email in attendees:
                if not re.match(email_pattern, email):
                    raise ValueError(f"Invalid email address: {email}")
                attendees_emails.append({"email": email})

        reminders_info = None
        if reminders is True:
            reminders_info = {"useDefault": True}
        elif isinstance(reminders, list):
            for reminder in reminders:
                if 'method' not in reminder or 'minutes' not in reminder:
                    raise ValueError("Each reminder must have 'method' and 'minutes' keys.")
                if reminder['method'] not in ['email', 'popup']:
                    raise ValueError("The reminder method must be 'email' or 'popup'.")
            reminders_info = {
                'useDefault': False,
                "overrides": reminders
            }
        else:
            reminders_info = {"useDefault": False}

        conference_data_info = None
        if conference_data:
            conference_data_info = {
                "createRequest": {
                    "requestId": str(uuid4()),
                    "conferenceSolutionKey": {
                        "type": "hangoutsMeet"
                    }
                }
            }

        event_body = {
            "summary": summary,
            "start": start,
            "end": end
        }

        if location:
            event_body["location"] = location

        if description:
            event_body["description"] = description

        if recurrence_data:
            event_body["recurrence"] = [recurrence_data]

        if attendees_emails:
            event_body["attendees"] = attendees_emails

        if reminders_info:
            event_body["reminders"] = reminders_info

        if conference_data_info:
            event_body["conferenceData"] = conference_data_info

        if color_id:
            event_body["colorId"] = color_id
        
        if transparency:
            event_body["transparency"] = transparency

        return event_body

    def _run(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        calendar_id: str = 'primary',
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None,
        attendees: Optional[List[str]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conference_data: Optional[bool] = None,
        color_id: Optional[str] = None,
        transparency: Optional[str] = None, 
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Run the tool to create an event in Google Calendar."""
        try:
            body = self._prepare_event(
                summary=summary,
                start_datetime=start_datetime,
                end_datetime=end_datetime,
                calendar_id=calendar_id,
                timezone=timezone,
                recurrence=recurrence,
                location=location,
                description=description,
                attendees=attendees,
                reminders=reminders,
                conference_data=conference_data,
                color_id=color_id, 
                transparency=transparency
            )

            conference_version = 1 if conference_data else 0

            event = self.api_resource.events().insert(
                calendarId=calendar_id,
                body=body,
                conferenceDataVersion=conference_version
            ).execute()

            return event.get('htmlLink')
        except Exception as error:
            raise Exception(f"An error occurred: {error}") from error

In [9]:
"""Search an event in Google Calendar.""" # NUEVO

from datetime import datetime
from typing import Any, Dict, List, Optional
from zoneinfo import ZoneInfo  # Python 3.9+

from pydantic import BaseModel, Field, field_validator

from langchain_core.callbacks import CallbackManagerForToolRun


class SearchEventsSchema(BaseModel):
    """Input for CalendarSearchEvents."""

    min_datetime: Optional[str] = Field(
        default_factory=lambda: datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        description=(
            "The start datetime for the events in 'YYYY-MM-DD HH:MM:SS' format. "
            f"The current year is {datetime.now().year}."
        )
    )
    max_datetime: Optional[str] = Field(
        ...,
        description="The end datetime for the events."
    )
    max_results: int = Field(
        default=10,
        description="The maximum number of results to return."
    )
    single_events: bool = Field(
        default=True,
        description=(
            "Whether to expand recurring events into instances and only return single "
            "one-off events and instances of recurring events."
            "'startTime' or 'updated'."
        )
    )
    order_by: str = Field(
        default="startTime",
        description="The order of the events, either 'startTime' or 'updated'."
    )
    query: Optional[str] = Field(
        default=None,
        description=(
            "Free text search terms to find events that match these terms in the following fields: "
            "summary, description, location, attendee's displayName, attendee's email, "
            "organizer's displayName, organizer's email."
        )
    )

    @field_validator('order_by')
    def validate_order_by(cls, v):
        if v not in ['startTime', 'updated']:
            raise ValueError("order_by must be 'startTime' or 'updated'")
        return v

    @field_validator('max_results')
    def validate_max_results(cls, v):
        if v <= 0:
            raise ValueError("max_results must be a positive integer")
        return v

    @field_validator('max_datetime')
    def validate_datetimes(cls, v, values):
        min_dt_str = values.get('min_datetime')
        if min_dt_str:
            min_dt = datetime.strptime(min_dt_str, "%Y-%m-%d %H:%M:%S")
            max_dt = datetime.strptime(v, "%Y-%m-%d %H:%M:%S")
            if max_dt < min_dt:
                raise ValueError('max_datetime must be after min_datetime')
        return v


class CalendarSearchEvents(GoogleCalendarBaseTool):
    """Tool that retrieves events from Google Calendar."""

    name: str = "search_events"
    description: str = "Use this tool to search events in the calendar."
    args_schema: Type[SearchEventsSchema] = SearchEventsSchema

    def _get_calendars_info(self) -> List[Dict[str, Any]]:
        """Get the calendars info."""
        calendars = self.api_resource.calendarList().list().execute()
        return calendars.get('items', [])

    def _get_calendar_timezone(self, calendars_info: List[Dict[str, Any]], calendar_id: str) -> Optional[str]:
        """Get the timezone of the current calendar."""
        for cal in calendars_info:
            if cal['id'] == calendar_id:
                return cal.get('timeZone')
        return None

    def _get_calendar_ids(self, calendars_info: List[Dict[str, Any]]) -> List[str]:
        """Get the calendar IDs."""
        return [cal['id'] for cal in calendars_info if cal.get('selected')]

    def _process_data_events(self, events_data: List[Dict[str, Any]]) -> List[Dict[str, Optional[str]]]:
        """Process the data events."""
        simplified_data = []
        for data in events_data:
            event_dict = {
                "id": data.get("id"),
                "htmlLink": data.get("htmlLink"),
                "summary": data.get("summary"),
                "creator": data.get("creator", {}).get("email"),
                "organizer": data.get("organizer", {}).get("email"),
                "start": data.get("start", {}).get("dateTime") or data.get("start", {}).get("date"),
                "end": data.get("end", {}).get("dateTime") or data.get("end", {}).get("date"),
            }
            simplified_data.append(event_dict)
        return simplified_data

    def _run(
        self,
        min_datetime: Optional[str],
        max_datetime: Optional[str],
        max_results: int = 10,
        single_events: bool = True,
        order_by: str = "startTime",
        query: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> List[Dict[str, Optional[str]]]:
        """Run the tool to search events in Google Calendar."""
        try:
            calendars_info = self._get_calendars_info()
            calendars = self._get_calendar_ids(calendars_info)
            events = []
            for calendar in calendars:
                tz_name = self._get_calendar_timezone(calendars_info, calendar)
                if tz_name:
                    calendar_tz = ZoneInfo(tz_name)
                else:
                    calendar_tz = None

                if min_datetime:
                    time_min = datetime.strptime(min_datetime, "%Y-%m-%d %H:%M:%S").astimezone(calendar_tz).isoformat()
                if max_datetime:
                    time_max = datetime.strptime(max_datetime, "%Y-%m-%d %H:%M:%S").astimezone(calendar_tz).isoformat()

                events_result = self.api_resource.events().list(
                    calendarId=calendar,
                    timeMin=time_min,
                    timeMax=time_max,
                    maxResults=max_results,
                    singleEvents=single_events,
                    orderBy=order_by,
                    q=query
                ).execute()
                cal_events = events_result.get('items', [])
                events.extend(cal_events)
            return self._process_data_events(events)
        except Exception as error:
            raise Exception(f"An error occurred while fetching events: {error}") from error

In [11]:
"""Update an event in Google Calendar.""" # NUEVO

from datetime import datetime
from typing import Any, Dict, List, Optional, Type, Union
from uuid import uuid4
import re

from pydantic import BaseModel, Field

from langchain_core.callbacks import CallbackManagerForToolRun


class UpdateEventSchema(BaseModel):
    """Input for CalendarUpdateEvent."""

    event_id: str = Field(
        ..., 
        description="The event ID to update."
    )
    calendar_id: str = Field(
        default="primary",
        description="The calendar ID to create the event in."
    )
    summary: Optional[str] = Field(
        default=None,
        description="The title of the event."
    )
    start_datetime: Optional[str] = Field(
        default=None,
        description=(
            "The new start datetime for the event in 'YYYY-MM-DD HH:MM:SS' format. "
            "If the event is an all-day event, set the time to 'YYYY-MM-DD' format."
        )
    )
    end_datetime: Optional[str] = Field(
        default=None,
        description=(
            "The new end datetime for the event in 'YYYY-MM-DD HH:MM:SS' format. "
            "If the event is an all-day event, set the time to 'YYYY-MM-DD' format."
        )
    )
    timezone: Optional[str] = Field(
        default=None,
        description="The timezone of the event."
    )
    recurrence: Optional[Dict[str, Any]] = Field(
        default=None,
        description=(
            "The recurrence of the event. "
            "Format: {'FREQ': <'DAILY' or 'WEEKLY'>, 'INTERVAL': <number>, "
            "'COUNT': <number or None>, 'UNTIL': <'YYYYMMDD' or None>, "
            "'BYDAY': <'MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU' or None>}. "
            "Use either COUNT or UNTIL, but not both; set the other to None."
        )
    )
    location: Optional[str] = Field(
        default=None,
        description="The location of the event."
    )
    description: Optional[str] = Field(
        default=None,
        description="The description of the event."
    )
    attendees: Optional[List[str]] = Field(
        default=None,
        description="A list of attendees' email addresses for the event."
    )
    reminders: Union[None, bool, List[Dict[str, Any]]] = Field(
        default=None,
        description=(
            "Reminders for the event. "
            "Set to True for default reminders, or provide a list like "
            "[{'method': 'email', 'minutes': <minutes>}, ...]. "
            "Valid methods are 'email' and 'popup'."
        )
    )
    conference_data: Optional[bool] = Field(
        default=None,
        description="Whether to include conference data."
    )
    color_id: Optional[str] = Field(
        default=None,
        description=(
            "The color ID of the event. None for default. "
            "'1': Lavender, '2': Sage, '3': Grape, '4': Flamingo, '5': Banana, "
            "'6': Tangerine, '7': Peacock, '8': Graphite, '9': Blueberry, "
            "'10': Basil, '11': Tomato."
        )
    )
    transparency: Optional[str] = Field(
        default=None, 
        description=(
            "User availability for the event."
            "transparent for available and opaque for busy."
        )
    )
    send_updates: Optional[str] = Field(
        default=None,
        description=(
            "Whether to send updates to attendees. "
            "Allowed values are 'all', 'externalOnly', or 'none'."
        )
    )


class CalendarUpdateEvent(GoogleCalendarBaseTool):
    """Tool that updates an event in Google Calendar."""

    name: str = "update_calendar_event"
    description: str = "Use this tool to update an event. "
    args_schema: Type[UpdateEventSchema] = UpdateEventSchema

    def _get_event(self, event_id: str, calendar_id: str = 'primary') -> Dict[str, Any]:
        """Get the event by ID."""
        event = self.api_resource.events().get(calendarId=calendar_id, eventId=event_id).execute()
        return event
    
    def _is_all_day_event(self, start_datetime: str, end_datetime: str) -> bool:
        """Check if the event is an all-day event."""
        date_format = "%Y-%m-%d"
        try:
            datetime.strptime(start_datetime, date_format)
            datetime.strptime(end_datetime, date_format)
            return True
        except ValueError:
            return False

    def _refactor_event(
        self,
        event: Dict[str, Any],
        summary: Optional[str] = None,
        start_datetime: Optional[str] = None,
        end_datetime: Optional[str] = None,
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None,
        attendees: Optional[List[str]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conference_data: Optional[bool] = None,
        color_id: Optional[str] = None, 
        transparency: Optional[str] = None
    ) -> Dict[str, Any]:
        """Refactor the event body."""

        if summary is not None:
            event['summary'] = summary

        try: 
            if start_datetime and end_datetime:
                if self._is_all_day_event(start_datetime, end_datetime):
                    event['start'] = {"date": start_datetime}
                    event['end'] = {"date": end_datetime}
                else:
                    datetime_format = "%Y-%m-%d %H:%M:%S"
                    timezone = timezone or event['start']['timeZone']
                    start_dt = datetime.strptime(start_datetime, datetime_format)
                    end_dt = datetime.strptime(end_datetime, datetime_format)
                    event['start'] = {
                        "dateTime": start_dt.astimezone().isoformat(),
                        "timeZone": timezone
                    }
                    event['end'] = {
                        "dateTime": end_dt.astimezone().isoformat(),
                        "timeZone": timezone
                    }
        except ValueError as error:
            raise ValueError("The datetime format is incorrect.") from error

        if (recurrence is not None) and (isinstance(recurrence, dict)):
            recurrence_items = [f"{k}={v}" for k, v in recurrence.items() if v is not None]
            event.update({'recurrence': ['RRULE:' + ';'.join(recurrence_items)]})

        if location is not None:
            event.update({'location': location})

        if description is not None:
            event.update({'description': description})

        if attendees is not None:
            attendees_emails = []
            email_pattern = r'^[^@]+@[^@]+\.[^@]+$'
            for email in attendees:
                if not re.match(email_pattern, email):
                    raise ValueError(f"Invalid email address: {email}")
                attendees_emails.append({"email": email})
            event.update({'attendees': attendees_emails})

        if reminders is not None:
            if reminders is True:
                event.update({'reminders': {"useDefault": True}})
            elif isinstance(reminders, list):
                for reminder in reminders:
                    if 'method' not in reminder or 'minutes' not in reminder:
                        raise ValueError("Each reminder must have 'method' and 'minutes' keys.")
                    if reminder['method'] not in ['email', 'popup']:
                        raise ValueError("The reminder method must be 'email' or 'popup'.")
                event.update({'reminders': {"useDefault": False, "overrides": reminders}})
            else:
                event.update({'reminders': {"useDefault": False}})

        if (conference_data is not None) and (conference_data == True):
                event.update({'conferenceData': {
                    "createRequest": {
                        "requestId": str(uuid4()),
                        "conferenceSolutionKey": {
                            "type": "hangoutsMeet"
                        }
                    }}
                })
        else:
            event.update({'conferenceData': None})

        if color_id is not None:
            event['colorId'] = color_id

        if transparency is not None:
            event.update({'transparency': transparency})

        return event

    def _run(
        self,
        event_id: str,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        calendar_id: str = 'primary',
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None,
        attendees: Optional[List[str]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conference_data: Optional[bool] = None,
        color_id: Optional[str] = None,
        transparency: Optional[str] = None, 
        send_updates: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Run the tool to update an event in Google Calendar."""
        try:
            event = self._get_event(event_id, calendar_id)
            body = self._refactor_event(
                event=event,
                summary=summary,
                start_datetime=start_datetime,
                end_datetime=end_datetime,
                calendar_id=calendar_id,
                timezone=timezone,
                recurrence=recurrence,
                location=location,
                description=description,
                attendees=attendees,
                reminders=reminders,
                conference_data=conference_data,
                color_id=color_id, 
                transparency=transparency
            )

            conference_version = 1 if conference_data else 0

            result = self.api_resource.events().update(
                calendarId=calendar_id,
                eventId=event_id,
                body=body,
                conferenceDataVersion=conference_version, 
                sendUpdates=send_updates
            ).execute()

            return result.get('htmlLink')
        except Exception as error:
            raise Exception(f"An error occurred: {error}") from error

In [ ]:
"""Move an event between calendars in Google Calendar.""" # NUEVO

from typing import Optional, Type

from pydantic import BaseModel, Field

from langchain_core.callbacks import CallbackManagerForToolRun


class MoveEventSchema(BaseModel):
    """Input for CalendarMoveEvent."""

    event_id: str = Field(
        ..., 
        description="The event ID to move."
    )
    origin_calenddar_id: str = Field(
        ..., 
        description="The origin calendar ID."
    )
    destination_calendar_id: str = Field(
        ...,
        description="The destination calendar ID."
    )
    send_updates: Optional[str] = Field(
        default=None,
        description=(
            "Whether to send updates to attendees."
            "Allowed values are 'all', 'externalOnly', or 'none'."
        )
    )


class CalendarMoveEvent(GoogleCalendarBaseTool):
    """Tool that move an event between calendars in Google Calendar."""

    name: str = "move_calendar_event"
    description: str = "Use this tool to move an event between calendars."
    args_schema: Type[MoveEventSchema] = MoveEventSchema
        
    def _run(
        self, 
        event_id: str, 
        origin_calendar_id: str, 
        destination_calendar_id: str,
        send_updates: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Run the tool to update an event in Google Calendar."""
        try: 
            result = self.api_resource.events().move(
                eventId=event_id,
                calendarId=origin_calendar_id,
                destination=destination_calendar_id,
                sendUpdates=send_updates
            ).execute()
            return result.get('htmlLink') 
        except Exception as error:
            raise Exception(f"An error occurred: {error}") from error

---

In [173]:
from __future__ import annotations

from typing import TYPE_CHECKING, List

from langchain_community.agent_toolkits.base import BaseToolkit
from langchain_core.tools import BaseTool
from pydantic import ConfigDict, Field

from langchain_google_community.gmail.utils import build_resource_service

if TYPE_CHECKING:
    # This is for linting and IDE typehints
    from googleapiclient.discovery import Resource  # type: ignore[import]
else:
    try:
        # We do this so pydantic can resolve the types when instantiating
        from googleapiclient.discovery import Resource
    except ImportError:
        pass


SCOPES = ["https://www.googleapis.com/auth/calendar"]

class GoogleCalendarToolkit(BaseToolkit):
    """Toolkit for interacting with GoogleCalendar."""

    api_resource: Resource = Field(default_factory=build_resource_service)

    model_config = ConfigDict(
        arbitrary_types_allowed=True,
    )

    def get_tools(self) -> List[BaseTool]:
        """Get the tools in the toolkit."""
        return [
            CalendarCreateEvent(api_resource=self.api_resource),
            CalendarSearchEvents(api_resource=self.api_resource)
        ]

In [174]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)


# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="../secrets/token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="../secrets/credentials.json",
)
api_resource = build_resource_service(credentials=credentials, service_name='calendar', service_version='v3')
toolkit = GoogleCalendarToolkit(api_resource=api_resource)

In [175]:
tools = toolkit.get_tools()
tools

[CalendarCreateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x128cac800>),
 CalendarGetEvents(api_resource=<googleapiclient.discovery.Resource object at 0x128cac800>)]

In [176]:
import os
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=os.getenv('OPENAI_MODEL'), api_key=os.getenv('OPENAI_API_KEY'))

In [177]:
model_with_tools = model.bind_tools(tools)

In [178]:
from langchain_core.messages import HumanMessage

prompt = "Crea un evento de todo el dia del evento de carros para hoy"

response = model_with_tools.invoke([HumanMessage(content=prompt)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'create_calendar_event', 'args': {'summary': 'Evento de Carros', 'start_datetime': '2024-10-25', 'end_datetime': '2024-10-26', 'attendees': None, 'reminders': True}, 'id': 'call_hwXb9H4ho8be2Ds9sj4xTEyz', 'type': 'tool_call'}]


---

In [179]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [184]:
example_query = "Haz una sesion de meet para mañana a las 10am"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for data in events:
    data["messages"][-1].pretty_print()

================================ Human Message =================================

Haz una sesion de meet para mañana a las 10am
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_aPLaSAlTX7AFabyZEbm4WL3w)
 Call ID: call_aPLaSAlTX7AFabyZEbm4WL3w
  Args:
    summary: Sesión de Meet
    start_datetime: 2024-10-26 10:00:00
    end_datetime: 2024-10-26 11:00:00
    attendees: None
    conference_data: True
================================= Tool Message =================================
Name: create_calendar_event

Error: Exception('An error occurred: The read operation timed out')
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_QbVja0chCo2P6oSkOiOQnnRO)
 Call ID: call_QbVja0chCo2P6oSkOiOQnnRO
  Args:
    summary: Sesión de Meet
    start_datetime: 2024-10-26 10:00:00
    end_datetime: 2024-10-26 11:00:00
    attendees: 